# BC/DR: Synchronize Account-Level Objects Between Primary and Standby (Snowflake)

This Snowflake Notebook sets up replication of account-level objects from a primary account to a standby account as part of BC/DR. It uses failover groups to replicate objects such as users, roles, warehouses, and more. It also shows how to schedule the replication frequency.

- Replace placeholders like `<ORG_NAME>`, `<PRIMARY_ACCOUNT_NAME>`, `<STANDBY_ACCOUNT_NAME>`, `<WAREHOUSE_NAME>`.
- Run cells with the appropriate role indicated in each section.

References:
- [Account Replication and Failover Overview](https://docs.snowflake.com/en/user-guide/account-replication-intro)
- [Replicating Databases and Account Objects Across Multiple Accounts](https://docs.snowflake.com/en/user-guide/account-replication-config)



## Prerequisites and context

- You need `ORGADMIN` for org-wide enablement and `ACCOUNTADMIN` in each account.
- Ensure network and security policies allow cross-account replication per your org standards.
- Replace placeholders like `<ORG_NAME>.<PRIMARY_ACCOUNT_NAME>` and `<ORG_NAME>.<STANDBY_ACCOUNT_NAME>`.

Useful docs:
- [Introduction to replication & failover](https://docs.snowflake.com/en/user-guide/replication-intro)
- [Account object replication considerations](https://docs.snowflake.com/en/user-guide/account-replication-considerations)


## Step 1: Enable replication features for accounts (run as ORGADMIN)

Enable replication for each Snowflake account involved. This allows account object/database replication across accounts in the org.

Docs:
- [SYSTEM$GLOBAL_ACCOUNT_SET_PARAMETER](https://docs.snowflake.com/en/sql-reference/functions/system_global_account_set_parameter)


In [ ]:
-- Role: ORGADMIN
-- Purpose: Enable replication for both accounts in the org
USE ROLE ORGADMIN;

-- Replace with your org and account names
SELECT SYSTEM$GLOBAL_ACCOUNT_SET_PARAMETER('<ORG_NAME>.<PRIMARY_ACCOUNT_NAME>', 'ENABLE_ACCOUNT_DATABASE_REPLICATION', 'true');
SELECT SYSTEM$GLOBAL_ACCOUNT_SET_PARAMETER('<ORG_NAME>.<STANDBY_ACCOUNT_NAME>', 'ENABLE_ACCOUNT_DATABASE_REPLICATION', 'true');


## Step 2: Create primary failover group for account objects (run in primary as ACCOUNTADMIN)

Use a failover group to replicate account-level objects and (optionally) databases to the standby account. You can also set an automatic replication schedule on the group.

Note: For `OBJECT_TYPES` and `ALLOWED_ACCOUNTS`, provide comma-separated lists without parentheses or single quotes (e.g., `OBJECT_TYPES = USERS, ROLES, WAREHOUSES`).

Docs:
- [Replicating databases and account objects across multiple accounts](https://docs.snowflake.com/en/user-guide/account-replication-config)
- [CREATE FAILOVER GROUP](https://docs.snowflake.com/en/sql-reference/sql/create-failover-group)


In [ ]:
-- Role: ACCOUNTADMIN (primary account)
USE ROLE ACCOUNTADMIN;
--drop failover group BCDR_ACCOUNT_OBJECTS;
CREATE FAILOVER GROUP BCDR_ACCOUNT_OBJECTS
  OBJECT_TYPES =  WAREHOUSES, ACCOUNT PARAMETERS, NETWORK POLICIES, DATABASES
  --, RESOURCE USERS, ROLES, MONITORS
  ALLOWED_DATABASES = <database name>
  ALLOWED_ACCOUNTS = <remote account name>
  REPLICATION_SCHEDULE = '10 MINUTE';  -- or a CRON, e.g., 'USING CRON 0 */1 * * * UTC';

## Alter Failover Group

Add or remove objects to a failover group


In [ ]:
ALTER FAILOVER GROUP BCDR_ACCOUNT_OBJECTS
  SET OBJECT_TYPES =  WAREHOUSES, ACCOUNT PARAMETERS, NETWORK POLICIES, DATABASES, USERS, ROLES;

In [ ]:
ALTER FAILOVER GROUP BCDR_ACCOUNT_OBJECTS
  SET allowed_databases = <comman separated list of databases names>;

## Standby steps moved to a separate notebook

Run standby procedures in: `BCDR_01_Standby_Account_Objects_Replica.ipynb`.

That notebook includes:
- Create replica failover group in standby
- Optional: Link account objects by name
- Initial refresh of the replica
- Verification

Docs:
- [CREATE FAILOVER GROUP](https://docs.snowflake.com/en/sql-reference/sql/create-failover-group)
- [ALTER FAILOVER GROUP ... REFRESH](https://docs.snowflake.com/en/sql-reference/sql/alter-failover-group)
- [SYSTEM$LINK_ACCOUNT_OBJECTS_BY_NAME](https://docs.snowflake.com/en/sql-reference/functions/system_link_account_objects_by_name)


## Step 6: Schedule replication frequency

You can schedule replication in two ways:

1) Use the failover group’s built-in `REPLICATION_SCHEDULE` (set in Step 2). Update it via `ALTER FAILOVER GROUP`.
2) Create a Snowflake task that periodically executes `ALTER FAILOVER GROUP ... REFRESH`.

Choose one approach that meets your RPO and operational model.

Docs:
- [CREATE/ALTER FAILOVER GROUP](https://docs.snowflake.com/en/sql-reference/sql/alter-failover-group)
- [Tasks: scheduling syntax & CRON](https://docs.snowflake.com/en/user-guide/tasks-intro)


In [ ]:
-- Option A: Adjust the built-in schedule on the failover group (primary account)
USE ROLE ACCOUNTADMIN;
ALTER FAILOVER GROUP BCDR_ACCOUNT_OBJECTS SET REPLICATION_SCHEDULE = '1440 MINUTE'; -- setting to 1 day
-- OR CRON format (example: top of every hour UTC):
--ALTER FAILOVER GROUP BCDR_ACCOUNT_OBJECTS SET REPLICATION_SCHEDULE = 'USING CRON 0 * * * * UTC';


In [ ]:
-- Option B: Create a task to refresh the failover group (primary account)
USE ROLE ACCOUNTADMIN;
CREATE OR REPLACE TASK BCDR_ACCOUNT_OBJECTS_REFRESH
  WAREHOUSE = <WAREHOUSE_NAME>
  SCHEDULE = '10 MINUTE'  -- or 'USING CRON 0 */1 * * * UTC'
  USER_TASK_TIMEOUT_MS = 3600000
AS
  ALTER FAILOVER GROUP BCDR_ACCOUNT_OBJECTS REFRESH;

ALTER TASK BCDR_ACCOUNT_OBJECTS_REFRESH RESUME;


## Step 7: Verify and monitor

Use the following commands to review failover groups and replication databases, and confirm schedules and last refresh times.

Docs:
- [SHOW FAILOVER GROUPS](https://docs.snowflake.com/en/sql-reference/sql/show-failover-groups)
- [SHOW REPLICATION DATABASES](https://docs.snowflake.com/en/sql-reference/sql/show-replication-databases)


In [ ]:
-- View failover groups and schedules (primary or standby)
SHOW FAILOVER GROUPS;

-- View replication databases (for visibility into database-level replication status)
--SHOW REPLICATION DATABASES;


## Step 8: (Optional) Configure client redirect

Client Redirect enables redirecting your client connections to Snowflake accounts in different regions without changing the connection settings for your application. 

NOTE: This can be done via SQL or via the UI

Docs:
- [REDIRECTING CLIENT CONNECTIONS](https://docs.snowflake.com/en/user-guide/client-redirect)

In [ ]:
-- Create a new primary connection
CREATE CONNECTION <connection name>;

-- View accounts in your organization that are enabled for replication
SHOW REPLICATION ACCOUNTS;

-- Configure failover accounts for the primary connection
ALTER CONNECTION <connection name>
  ENABLE FAILOVER TO ACCOUNTS <allowed accounts>;

-- View the details for the connection
SHOW CONNECTIONS;

In [ ]:
-- cleanup if needed
-- drop failover group BCDR_ACCOUNT_OBJECTS;